## Checking for GPU

In [1]:
!nvidia-smi

Wed Jun 30 21:28:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.11       Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8     2W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installing Ktrain library

In [3]:
!pip install ktrain

  Using cached scikit_learn-0.23.2-cp38-cp38-win_amd64.whl (6.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.6.0 requires scikit-learn>=0.24.0, but you have scikit-learn 0.23.2 which is incompatible.
pmdarima 1.8.2 requires numpy~=1.19.0, but you have numpy 1.21.0 which is incompatible.
imbalanced-learn 0.8.0 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


  Created wheel for ktrain: filename=ktrain-0.26.4-py3-none-any.whl size=25282450 sha256=6022531aeda87dae8a2be3e9e2e8ddb86fe3e34d9ae71c8220c4b230288b1fd2
  Stored in directory: c:\users\aman\appdata\local\pip\cache\wheels\82\70\fe\0583ec687b65e71c4de1ba3784dfe49ed7c3811baed1da5b4a
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9918 sha256=286dbeefb882d83933994454710a30491e232423f6e0b1b94574741d9327b951
  Stored in directory: c:\users\aman\appdata\local\pip\cache\wheels\58\e5\9e\2f94312239a6753267da6ceafe12d614603b78d308676df5e2
  Created wheel for keras-bert: filename=keras_bert-0.88.0-py3-none-any.whl size=34205 sha256=bcde239a416a528cc63f0dcc3414fc97dfabf78325196e978eb255d5356ec6b8
  Stored in directory: c:\users\aman\appdata\local\pip\cache\wheels\93\49\c0\a457bd5fa37d67ef12409478d387b6d06ff1e7621318d2f605
  Created wheel for keras-transformer: filename=keras_transformer-0.39.0-py3-none-any.whl size=12840 sha256=c2011e05e4fd3811ee436c199994de03eec16181691

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.4.1'

In [8]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,f1_score,roc_curve

In [10]:
train_df = pd.read_csv("Data/train.csv")
val_df = pd.read_csv("Data/val.csv")
test_df = pd.read_csv("Data/test.csv")
train_df.head()

,id,text,label
0,0,we propose deep network models and learning al...,3
1,1,multi-distance information computed by the MDL...,3
2,2,traditional solutions consider dense pedestria...,2
3,3,"in this paper, is used the lagrangian classica...",2
4,4,the aim of this work is to determine how vulne...,3


In [11]:
trainplusval=pd.concat([train_df,val_df])

In [12]:
trainplusval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34200 entries, 0 to 2699
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      34200 non-null  int64 
 1   text    34200 non-null  object
 2   label   34200 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ MB


In [13]:
word_count=test_df['text'].apply(lambda x:len(x.split()))

In [14]:
max(word_count)

91

In [15]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=trainplusval,
                                                                   text_column ='text',
                                                                   label_columns ='label',
#                                                                   val_df = val_df,
                                                                   maxlen = 91,
                                                                   preprocess_mode = 'bert')

['label_0', 'label_1', 'label_2', 'label_3']
       label_0  label_1  label_2  label_3
15207      0.0      0.0      0.0      1.0
18954      0.0      0.0      0.0      1.0
11033      0.0      0.0      0.0      1.0
25026      0.0      1.0      0.0      0.0
14958      1.0      0.0      0.0      0.0
['label_0', 'label_1', 'label_2', 'label_3']
       label_0  label_1  label_2  label_3
13639      0.0      1.0      0.0      0.0
9404       0.0      0.0      0.0      1.0
30188      0.0      0.0      0.0      1.0
15464      0.0      0.0      0.0      1.0
26221      0.0      0.0      0.0      1.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [16]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 91
done.


In [17]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

In [18]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
  22/5130 [..............................] - ETA: 47:56 - loss: 1.3380 - accuracy: 0.5844

KeyboardInterrupt: 

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['label_0', 'label_1', 'label_2', 'label_3']

In [ ]:
pred_val = predictor.predict(val_df['text'].values)

In [ ]:
prediction_val=[]
for pred in pred_val:
  if pred=='label_0':
    prediction_val.append(0)
  elif pred=='label_1':
    prediction_val.append(1)
  elif pred=='label_2':
    prediction_val.append(2)
  else:
    prediction_val.append(3)

In [ ]:
y_val=val_df['label']

In [ ]:
# Getting F1 & Accuracy score of validation predictions
f1 = f1_score(y_val, prediction_val,average='weighted')
accuracy = accuracy_score(y_val, prediction_val)
f1,accuracy

(0.893040682122157, 0.894074074074074)

##  Making Submission

In [ ]:
pred_val = predictor.predict(test_df['text'].values)

In [ ]:
prediction_test=[]
for pred in pred_val:
  if pred=='label_0':
    prediction_test.append(0)
  elif pred=='label_1':
    prediction_test.append(1)
  elif pred=='label_2':
    prediction_test.append(2)
  else:
    prediction_test.append(3)

In [ ]:
# Putting the results into the column of test dataset
test_df['label'] = prediction_test
test_df

,id,text,label
0,0,we propose a lightweight framework to detect i...,3
1,1,the proposed method presents an alternate solu...,2
2,2,proposed ear identification method fusing SIFT...,3
3,3,a method to reconstruct the three-dimensional ...,3
4,4,strong local consistencies can improve their p...,0
...,...,...,...
10795,10795,whole-body gradient echo scans of 240 subjects...,3
10796,10796,we present a tracker that accomplishes trackin...,3
10797,10797,the most popular FL algorithm is Federated Ave...,1
10798,10798,"in the field of Autonomous Driving, the system...",2


## Saving in CSV format

In [ ]:
import os

In [ ]:
# Saving out results in submission.csv
!mkdir assets
test_df.to_csv(os.path.join("assets", 'submission.csv'), index=False)